In [290]:
import os
from pathlib import Path

import jams
import pandas as pd

import pickle

In [291]:
# --- PATHS ---
PROJECT_ROOT = Path("..")
GUITARSET_ROOT = PROJECT_ROOT / "data" / "guitarset"

AUDIO_DIR = GUITARSET_ROOT / "audio_mono-pickup_mix"
ANNOTATION_DIR = GUITARSET_ROOT / "annotation"

# Validate directories exist (helps debugging)
assert AUDIO_DIR.exists(), f"Missing audio dir: {AUDIO_DIR}"
assert ANNOTATION_DIR.exists(), f"Missing annotation dir: {ANNOTATION_DIR}"

In [292]:
jams_files = []

for f in os.listdir(ANNOTATION_DIR):
    if "comp" in f:
        full_path = os.path.join(ANNOTATION_DIR, f)
        jams_files.append(full_path)

print("Total JAMS files:", len(jams_files))
print(jams_files[0])

Total JAMS files: 180
../data/guitarset/annotation/03_SS1-100-C#_comp.jams


In [293]:
jam = jams.load(jams_files[0])
print(jam.file_metadata)

{
  "title": "03_SS1-100-C#_comp",
  "artist": "",
  "release": "",
  "duration": 28.80000000000001,
  "identifiers": {},
  "jams_version": "0.3.1"
}


In [294]:
# --- CIRCLE OF FIFTHS ---
FLAT_MAJOR_KEYS  = {"F", "Bb", "Eb", "Ab", "Db", "Gb", "Cb"}
SHARP_MAJOR_KEYS = {"G", "D", "A", "E", "B", "F#", "C#"}

# --- NOTE → PITCH CLASS ---
NOTE_TO_SEMITONE = {
    "C": 0, "B#": 0,
    "C#": 1, "Db": 1,
    "D": 2,
    "D#": 3, "Eb": 3,
    "E": 4, "Fb": 4,
    "F": 5, "E#": 5,
    "F#": 6, "Gb": 6,
    "G": 7,
    "G#": 8, "Ab": 8,
    "A": 9,
    "A#": 10, "Bb": 10,
    "B": 11, "Cb": 11,
}

SEMITONE_TO_SHARP = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
SEMITONE_TO_FLAT  = ["C","Db","D","Eb","E","F","Gb","G","Ab","A","Bb","B"]

In [295]:
def relative_major_tonic(minor_tonic: str) -> str:
    """
    Convert natural minor tonic -> relative major tonic.
    Example: 'E' -> 'G'
    """
    pc = NOTE_TO_SEMITONE[minor_tonic]
    major_pc = (pc + 3) % 12

    return SEMITONE_TO_SHARP[major_pc] if minor_tonic in SHARP_MAJOR_KEYS else SEMITONE_TO_FLAT[major_pc]

def key_prefers_flats(key_mode: str) -> bool:
    """
    Decide whether the key signature prefers flat spellings.
        - key_mode format: 'Eb:major' or 'E:minor'
    """
    tonic_str, mode_str = key_mode.split(":")
    tonic = tonic_str.strip()
    mode = mode_str.strip().lower()

    if mode == "minor":
        tonic = relative_major_tonic(tonic)

    if tonic in FLAT_MAJOR_KEYS:
        return True
    if tonic in SHARP_MAJOR_KEYS:
        return False

    # Neutral keys (C major / A minor)
    return False

def normalize_chord_based_on_key(root: str, key_mode: str) -> str:
    """
    Normalize enharmonic spelling of `root` to match `key_mode`.
    Returns the (possibly unchanged) root name.
    """
    root = root.strip()

    if root not in NOTE_TO_SEMITONE:
        raise ValueError(f"Unknown note name: {root}")

    pc = NOTE_TO_SEMITONE[root]
    prefer_flats = key_prefers_flats(key_mode)

    preferred = SEMITONE_TO_FLAT[pc] if prefer_flats else SEMITONE_TO_SHARP[pc]

    # If the root is already spelled in a way consistent with the key, keep it
    if root == preferred:
        return root

    # Otherwise return normalized spelling
    return preferred

In [296]:
def normalize_chord_annotations(label, key_mode):
    # TODO:
    #  - Add 7ths to the chord
    #  - No default to major

    """
    Normalize GuitarSet chord labels to triads.

    Parameters
    ----------
    label : str
        expected format: root:quality(...)/bass

    Returns
    -------
    str
        root:maj | root:min | root:dim | root:hdim | root:aug | N
    """

    if label == "N" or ":" not in label:
            return "N"

    root, rest = label.split(":", 1)

    # normalize enharmonic root name
    root = normalize_chord_based_on_key(root, key_mode)

    rest = rest.lower().strip()
    q = "maj"
    if rest.startswith("min"):
        q = "min"
    elif rest.startswith("dim"):
        q = "dim"
    elif rest.startswith("aug"):
        q = "aug"
    elif rest.startswith("hdim"):
        q = "hdim"

    return f"{root}:{q}"

In [297]:
def extract_chord_annotations(jam, key_mode, normalized=True):
    """
    Extract performed chord annotations from a GuitarSet JAMS object.

    Parameters
    ----------
    jam : jams.JAMS
        A JAMS object containing GuitarSet annotations.

    key_mode : str
        Key mode of the excerpt, e.g., "Eb:minor"

    normalized : bool, optional
        If True (default), normalize chords to triads (maj/min/dim/aug).
        If False, keep the original chord labels from the JAMS file.

    Returns
    -------
    List[Dict[str, Any]]
        A list of chord segments, where each segment is represented as a dictionary:
            - 'start': float, start time in seconds
            - 'end': float, end time in seconds
            - 'label': str, chord label (normalized or original)
    """

    # Use the second "chord" annotation namespace (performed chords)
    chord_ann = jam.annotations.search(namespace="chord")[1]
    chords = []

    for c in chord_ann:
        label = c.value
        if normalized:
            label = normalize_chord_annotations(label, key_mode)

        chords.append({
            "start": c.time,
            "end": c.time + c.duration,
            "label": label,
        })

    return chords

In [298]:
def extract_beat_position(jam):
    """
    Extract beat_position annotations from a GuitarSet JAMS object.

    Parameters
    ----------
    jam : jams.JAMS
        A JAMS object containing GuitarSet annotations.

    Returns
    -------
    List[Dict[str, Any]]
        A list of beat_position segments, where each segment is represented as a dictionary:
            - 'time': float, start time in seconds
            - 'duration': float, duration of the beat in seconds
            - 'measure': int, the measure number
            - 'position': int, the position within the measure (in beats)
            - 'time_signature': str, e.g., "4/4", "6/8", "2/2"
    """
    beat_position_ann = jam.annotations.search(namespace="beat_position")
    if not beat_position_ann:
        return []

    beat_position_data = beat_position_ann[0].data
    beat_positions = []

    for bp in beat_position_data:
        value = bp.value
        beat_positions.append({
            "time": bp.time,
            "duration": bp.duration,
            "measure": value['measure'],
            "position": value['position'],
            "time_signature": f"{value['num_beats']}/{value['beat_units']}"
        })

    return beat_positions

In [299]:
def extract_key_mode(jam):
    """
    Extract key_mode annotations from a GuitarSet JAMS object.

    Parameters
    ----------
    jam : jams.JAMS
        A JAMS object containing GuitarSet annotations.

    Returns
    -------
    List[Dict[str, Any]]
        A list of key_mode segments, where each segment is represented as a dictionary:
            - 'value': str, key_mode of the excerpt
            - 'tonic': str, home pitch of the excerpt
            - 'mode': str, a scale that being used
    """
    key_mode_ann = jam.annotations.search(namespace="key_mode")
    if not key_mode_ann:
        return []

    key_mode_data = key_mode_ann[0].data
    key_modes = []

    for km in key_mode_data:
        tonic, mode = km.value.split(":")
        key_modes.append({
            "value": km.value,
            "tonic": tonic,
            "mode": mode,
        })

    if len(key_modes) != 1:
        print("WARNING: multiple key_mode annotations found!")

    return key_modes

In [300]:
dataset_index = []

for jams_path in jams_files:
    jam = jams.load(jams_path)

    basename = os.path.basename(jams_path).replace(".jams", "")
    audio_path = os.path.join(AUDIO_DIR, basename + "_mix" + ".wav")

    if not os.path.exists(audio_path):
        continue

    key_mode_collection = extract_key_mode(jam)

    dataset_index.append({
        "id": basename,
        "audio_path": audio_path,
        "jams_path": jams_path,
        "chords": extract_chord_annotations(jam, key_mode_collection[0]["value"], True),
        "beat_position": extract_beat_position(jam),
        "key_mode": key_mode_collection,
    })

print("Total usable tracks:", len(dataset_index))

Total usable tracks: 180


In [301]:
songs = sorted(dataset_index, key=lambda s: s["id"])

with pd.ExcelWriter("../data/xlsx_excerpts/excerpts.xlsx", engine="openpyxl") as writer:
    for i, song in enumerate(songs):
        meta = {
            "id": song["id"],
            "audio_path": song["audio_path"],
            "jams_path": song["jams_path"],
        }

        chords_df = pd.DataFrame(song["chords"])
        beats_df = pd.DataFrame(song["beat_position"])
        key_modes_df = pd.DataFrame(song["key_mode"])

        # make sure both have the same length by outer-joining on index
        merged = pd.concat(
            [key_modes_df, chords_df, beats_df],
            axis=1
        )

        # add meta-columns (repeat automatically)
        for k, v in meta.items():
            merged[k] = v

        # reorder columns (meta first)
        cols = ["id", "audio_path", "jams_path"] + \
               [c for c in merged.columns if c not in ("id","audio_path","jams_path")]
        merged = merged[cols]

        sheet = f"{song['id']}"
        merged.to_excel(writer, index=False, sheet_name=sheet)

        # autosize columns
        ws = writer.sheets[sheet]
        for col in ws.columns:
            max_len = 0
            col_letter = col[0].column_letter
            for cell in col:
                try:
                    val = str(cell.value)
                except:
                    val = ""
                max_len = max(max_len, len(val))
            ws.column_dimensions[col_letter].width = max_len + 2

In [302]:
from pathlib import Path

OUT = Path("../data/dataset/dataset_index.pkl")

with open(OUT, "wb") as f:
    pickle.dump(dataset_index, f)

print("Saved to:", OUT)


Saved to: ../data/dataset/dataset_index.pkl
